In [3]:
import pandas as pd
import numpy as np
import json 
import os
import torch

In [21]:
print(os.getcwd())
style_fake_data = pd.read_json("./gossipcop_v3_keep_data_in_proper_length.json",orient='index')
type(style_fake_data[:20].index)

/mnt/c/Users/1/PycharmProjects/CSFND_yanjie/gossipcop_dataset


pandas.core.indexes.base.Index

In [14]:
style_fake_data[style_fake_data.label=='fake'].count()[0], style_fake_data[style_fake_data.label=='real'].count()[0]

/tmp/ipykernel_28582/274957224.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  style_fake_data[style_fake_data.label=='fake'].count()[0], style_fake_data[style_fake_data.label=='real'].count()[0]
/tmp/ipykernel_28582/274957224.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  style_fake_data[style_fake_data.label=='fake'].count()[0], style_fake_data[style_fake_data.label=='real'].count()[0]


(3784, 11945)

In [16]:
def get_all_top_image_id(image_dir_path):
    suffix_to_remove = '_top_img.png'
    all_items  = os.listdir(image_dir_path)
    top_image_ids = {item.rstrip(suffix_to_remove) for item in all_items if os.path.isfile(os.path.join(image_dir_path, item)) and item.endswith(suffix_to_remove)}
    return top_image_ids

def filter_data(data,image_dir_path):
    top_image_ids = get_all_top_image_id(image_dir_path)
    print(len(top_image_ids))
    has_top_img_list = [id in top_image_ids for id in data.id]
    return data[has_top_img_list]
    
filter_data = filter_data(style_fake_data,'top_img')
# 2.只保留ID  文本内容 标签
keep_features = ["id","text","label"]
keep_data = filter_data[keep_features]



4699


,id,text,label
gossipcop-941805037,gossipcop-941805037,Thousands are taking the streets to protest Pr...,fake
gossipcop-9588339534,gossipcop-9588339534,"Many stars saw great success in 2016, but the ...",fake
gossipcop-9878194459,gossipcop-9878194459,So while the internet is busy obsessing over J...,fake
gossipcop-9521617242,gossipcop-9521617242,Next stop on the 2018 award season train? The ...,fake
gossipcop-8996311550,gossipcop-8996311550,A new survey conducted by the British Heart Fo...,fake
...,...,...,...
gossipcop-899716,gossipcop-899716,Maeve McDermott\n\nUSA TODAY\n\nJada Pinkett S...,real
gossipcop-910016,gossipcop-910016,"TechCrunch\n\nOmidyar Network, backed by eBay ...",real
gossipcop-907293,gossipcop-907293,Gigi & Zayn Gave Us the Best Idea For a Valent...,real
gossipcop-941486,gossipcop-941486,Jada Pinkett Smith recalled that heartbreaking...,real


In [19]:
keep_data.text.map(lambda x: len(x)).mean()

2434.691423707172

In [23]:
def split_data(data):
    # 3. 保存到文件中，文本和图像一一对应。
    # 划分训练和测试数据集，要求训练集和测试集中的正负类的分布差不多，
    # 其中 文本 ID 和图像ID一致，也对应上文本的存储位置。
    df_shuffled = data.sample(frac=1.0, random_state=42)
    
    # 计算训练集和测试集的数量
    num_train_samples = 80
    num_val_samples = 80
    num_test_samples = len(df_shuffled) - 160
    
    
    # 分割数据集
    
    train_set = df_shuffled.iloc[:num_train_samples]
    val_set = df_shuffled.iloc[num_train_samples:num_train_samples + num_val_samples]
    test_set = df_shuffled.iloc[num_train_samples + num_val_samples:]
    return train_set, val_set, test_set


data_real_news = keep_data[keep_data.label=='real']
data_fake_news = keep_data[keep_data.label=='fake']
real_train_data ,real_val_data ,real_test_data = split_data(data_real_news)
fake_train_data ,fake_val_data ,fake_test_data = split_data(data_fake_news)


train_set = pd.concat([real_train_data,fake_train_data]).sample(frac=1.0, random_state=42)
val_set = pd.concat([real_val_data,fake_val_data]).sample(frac=1.0, random_state=42)
test_set = pd.concat([real_test_data,fake_test_data]).sample(frac=1.0, random_state=42)


train_set[:5], val_set[:5], test_set[:5]

(                                        id  \
 gossipcop-9541238456  gossipcop-9541238456   
 gossipcop-9905143937  gossipcop-9905143937   
 gossipcop-9713275683  gossipcop-9713275683   
 gossipcop-930681          gossipcop-930681   
 gossipcop-9186072396  gossipcop-9186072396   
 
                                                                    text label  
 gossipcop-9541238456  Courtesy of Instagram\n\nWhoa! A new report is...  fake  
 gossipcop-9905143937  Justin is "absolutely" a pot smoker, he said i...  fake  
 gossipcop-9713275683  Advertisement\n\nThere are a million reasons t...  fake  
 gossipcop-930681      Jessica Alba is one grateful birthday girl!\n\...  real  
 gossipcop-9186072396  Season 14 of NBC’s hit singing reality competi...  fake  ,
                                         id  \
 gossipcop-9446264931  gossipcop-9446264931   
 gossipcop-9983761608  gossipcop-9983761608   
 gossipcop-9311146319  gossipcop-9311146319   
 gossipcop-941658          gossipcop-9416

In [24]:
train_set.to_json("./gossipcop_v3_keep_data_in_proper_length_train.json", orient="records", indent=4,)

In [25]:
val_set.to_json("./gossipcop_v3_keep_data_in_proper_length_valid.json", orient="records", indent=4,)

In [26]:
test_set.to_json("./gossipcop_v3_keep_data_in_proper_length_test.json", orient="records", indent=4,)

In [11]:
img_list_path = df_train["origin_id"].map(lambda x: img_dir+"/" + str(x)+"_top_img.png").tolist()